<a href="https://colab.research.google.com/github/gundall98/AI-X-MachineRunning/blob/master/AI%2BX_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 웹 크롤링을 통한 KOSPI,투자자별 매매 동향 데이터 불러오기
## 데이터 출처: 네이버 금융
### 저자: 2017056617 파이낸스경영학과 박건 

* 인터넷 속도에 따라 런닝 타임이 변동될 수 있음


In [97]:

from bs4 import BeautifulSoup
import urllib.request as req
import pandas as pd

Fdata = []
Inddata = []
Insdata = []
Ddata = []
KOSPIs=[]
dates = []

# 매매동향 기준 페이지당 10개의 데이터
# 코스피 기준 페이지당 6개의 데이터
num = input("확인할 데이터의 수를 입력하시오: ")


# -----------------------------------------------#
#              투자자별 매매동향                 #
# -----------------------------------------------#
pages = int(num)//10
left = int(num)%10

for i in range(1,pages+1):
  url = "https://finance.naver.com/sise/investorDealTrendDay.nhn?bizdate=2020060404&sosok=&page="+str(i)


  response = req.urlopen(url)
  soup = BeautifulSoup(response, 'html.parser') 

  body = soup.find('body')
  tr = body.find_all('tr')

  for j in tr:
    date = j.find('td',{'class':'date2'})
    if date != None:
      date= j.find_all('td')[:4][0].text
      Foreign = j.find_all('td')[:4][2].text
      Individual = j.find_all('td')[:4][1].text
      Institution = j.find_all('td')[:4][3].text
      Ddata.insert(0,date)
      Fdata.insert(0,Foreign)
      Inddata.insert(0,Individual)
      Insdata.insert(0,Institution)

# 잔여 데이터 불러오기 (입력한 요구 데이터 수량 % 크롤링한 페이지 수)
if left != 0:
  url = "https://finance.naver.com/sise/investorDealTrendDay.nhn?bizdate=2020060404&sosok=&page="+str(pages+1)

  response = req.urlopen(url)
  soup = BeautifulSoup(response, 'html.parser') 

  body = soup.find('body')
  tr = body.find_all('tr')
  count = 0

  for j in tr:
    if count < left:
      date = j.find('td',{'class':'date2'})
      if date != None:

        date= j.find_all('td')[:4][0].text
        Foreign = j.find_all('td')[:4][2].text
        Individual = j.find_all('td')[:4][1].text
        Institution = j.find_all('td')[:4][3].text
        Ddata.insert(0,date)
        Fdata.insert(0,Foreign)
        Inddata.insert(0,Individual)
        Insdata.insert(0,Institution)
        count += 1

# -----------------------------------------------#
#              KOSPI 종가 데이터                 #
# -----------------------------------------------#
pages = int(num)//6
left = int(num)%6

for i in range(1,pages+1):
  url = "https://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI&page="+str(i)

  response = req.urlopen(url)
  soup = BeautifulSoup(response, 'html.parser') 

  body = soup.find('body')
  tr = body.find_all('tr')

  for j in tr:
    date = j.find('td',{'class':'date'})
    if date != None:
      date= j.find_all('td')[:4][0].text
      if date not in dates:
        
        dates.append(date)

        KOSPI = j.find_all('td')[:4][1].text
        
        KOSPIs.insert(0,KOSPI)

# 잔여 데이터 불러오기 (입력한 요구 데이터 수량 % 크롤링한 페이지 수)
if left != 0:
  url = "https://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI&page="+str(pages+1)

  response = req.urlopen(url)
  soup = BeautifulSoup(response, 'html.parser') 

  body = soup.find('body')
  tr = body.find_all('tr')
  count = 0

  for j in tr:
    if count < left:
      date = j.find('td',{'class':'date'})
      if date != None:
        date= j.find_all('td')[:4][0].text
        if date not in dates:
          
          dates.append(date)

          KOSPI = j.find_all('td')[:4][1].text
          
          KOSPIs.insert(0,KOSPI)
          count += 1
    
# -----------------------------------------------#
#           pandas 이용 데이터 정리              #
# -----------------------------------------------#
table = pd.DataFrame(index = Ddata)
table["개인투자자"] = Inddata
table["외국인"] = Fdata
table["기관"] = Insdata
table["KOSPI"] = KOSPIs
table

확인할 데이터의 수를 입력하시오: 100


,개인투자자,외국인,기관,KOSPI
19.11.11,"1,405","-1,722",-9,"2,124.09"
19.11.12,238,-395,-151,"2,140.92"
19.11.13,"2,277",-280,"-2,419","2,122.45"
19.11.14,-620,"-2,087","2,261","2,139.23"
19.11.15,"-6,561","-1,416","7,859","2,162.18"
...,...,...,...,...
20.03.30,"1,986","-4,272","2,236","1,717.12"
20.03.31,"3,845",-711,"-2,901","1,754.64"
20.04.01,"11,508","-5,752","-6,199","1,685.46"
20.04.02,"2,700","-6,233","3,599","1,724.86"


,개인투자자,외국인,기관,KOSPI
16.03.10,"-1,219","6,503","-5,820","1,969.33"
16.03.11,-568,"1,475","-1,501","1,971.41"
16.03.14,"1,352",819,"-2,450","1,972.27"
16.03.15,95,631,"-1,314","1,969.97"
16.03.16,-459,976,-908,"1,974.90"
...,...,...,...,...
20.03.30,"1,986","-4,272","2,236","1,717.12"
20.03.31,"3,845",-711,"-2,901","1,754.64"
20.04.01,"11,508","-5,752","-6,199","1,685.46"
20.04.02,"2,700","-6,233","3,599","1,724.86"
